Pour se connecter à ce notebook 
Besoin de lancer  l'image singularity de : _Containers/shortcakelight.sif

https://github.com/rnakato/ShortCake/tree/master
Est conçu spécifiquement pour les analyses de données single cell

Pour lancer l'image singularity et pouvoir se connecter aux notebooks jupyter:
srun --pty bash
singularity exec 6_Containers/shortcakelight.sif jupyternotebook.sh

puis faire le lien ssh sur l'ordi avec cette commande  que l'on ecrit dans le terminal local: 
ssh -A -t -t vgoupille@genossh.genouest.org -L 8888:localhost:8888 ssh cl1n030 -L 8888:localhost:8888
modifier le port (numero et noeud si besoin)

si probleme de connection 

ls -a
rm -r .vscode-server
puis se remet sur vscode 
et refait la liaison avec ssh remote (et reinstaller les extensions...)

Library strategy: microSPLiT
The data preprocessing and alignment was performed using a modified SPLiT-seq pipeline (https://github.com/Alex-Rosenberg/split-seq-pipeline). Modifications: using STAR with the splicing isoform detection swiwwe usededt the hioutghoutest-scored multimapping re. We also keptwening a ctional count based on the number igninggood alignments, since bacterial genomes are known to contain overlapping CDSs.
Genome_build: ASM904v1.45 and ASM80076v1.37 from EnsemblBacteria
Supplementary_files_format_and_content: Cell by gene matrix (with cells above threshold 200 UMI/cell); Gene names; Cell annotations (barcode and well for heat shock data, OD for B.subtilis growth curve). For B. Subtilis growth curve, only mRNA are included in the cell-by-gene matrix.
Supplementary_files_format_and_content: For the heat-shock data, barcodes in wells 1-24 belong to heat-shocked cells and in wells 25-48 belong to control cells.
 	

Ce texte décrit les stratégies et les méthodes utilisées pour traiter et aligner les données issues de l’expérience scRNA-seq réalisée avec la méthode microSPLiT.
	1.	Stratégie de séquençage (Library strategy) :
	•	La méthode utilisée est microSPLiT, qui est une adaptation de la technique SPLiT-seq, permettant de réaliser du scRNA-seq sans nécessiter d’isolation physique des cellules.
	2.	Prétraitement et alignement des données :
	•	L’alignement des séquences a été effectué avec une version modifiée du pipeline SPLiT-seq disponible sur GitHub (https://github.com/Alex-Rosenberg/split-seq-pipeline).
	•	Modifications apportées :
	•	Utilisation de STAR (un logiciel d’alignement des lectures RNA-seq) avec une détection des isoformes d’épissage.
	•	Sélection des meilleurs alignements multimapping selon un critère de score élevé.
	•	Conservation d’un compte fonctionnel basé sur le nombre d’alignements de bonne qualité, ce qui est pertinent pour les génomes bactériens où plusieurs CDSs (séquences codantes) peuvent se chevaucher.
	3.	Références génomiques utilisées :
	•	Les génomes de référence utilisés pour l’alignement sont :
	•	ASM904v1.45
	•	ASM80076v1.37
	•	Ces versions proviennent de la base de données EnsemblBacteria.
	4.	Fichiers supplémentaires et leur contenu :
	•	Les fichiers supplémentaires fournis incluent :
	•	Matrice cellule-gène (contenant uniquement les cellules ayant un minimum de 200 UMI/cellule).
	•	Noms des gènes.
	•	Annotations des cellules :
	•	Pour les données du stress thermique (heat-shock) : les cellules sont annotées par code-barres et puits.
	•	Pour la courbe de croissance de Bacillus subtilis : les cellules sont annotées selon leur densité optique (OD). Seuls les ARNm (mRNA) sont inclus dans la matrice cellule-gène.
	5.	Organisation des cellules dans l’expérience de stress thermique :
	•	Les cellules sont disposées dans 48 puits, avec une distinction entre conditions :
	•	Puits 1-24 : cellules soumises au stress thermique.
	•	Puits 25-48 : cellules témoins (non soumises au stress thermique).

replica 2 : M14 => B. subtilis PY79 (OD0.5-OD3.2)

replica 1 : M15 => B. subtilis PY79 (OD0.5-OD6.0)

other plate : M11 =>	B. subtilis PY79 + E. coli MW1255

# Telechargement des données preocessées  de l'article de Kuchina et al: 
Microbial single-cell RNA sequencing by split-pool barcoding
https://doi.org/10.1126/science.aba5257

protocole 2024: 
https://doi.org/10.1038/s41596-024-01007-w

les données de séquençage brute sont disponible sur SRA : 
The raw sequencing files are available at the Sequence Read Archive:
    - GSM4594094 : https://www.ncbi.nlm.nih.gov/sra/SRX8485151%5baccn%5d
    - GSM4594095 : https://www.ncbi.nlm.nih.gov/sra/SRX8485152%5baccn%5d
    - GSM4594096 : https://www.ncbi.nlm.nih.gov/sra/SRX8485153%5baccn%5d

Processed data were submitted to Gene Expression Omnibus, with accession number GSE151940.

#Les données processées sont disponibles sur le site de GEO:
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE151940


SUPPLEMENTARY MATERIALS 
https://www.science.org/doi/10.1126/science.aba5257


NOUS N'ALLONS PAS UTILISER C'EST DONNEES QUI NE SONT PAS OBTENUE AVEC STARsolo 



Un autre article a analysé les données de scRNA-seq de la même expérience.
"Single-cell heterogeneity in ribosome content and the  consequences for the growth laws" par Brettner et al. (2024).

https://pmc.ncbi.nlm.nih.gov/articles/PMC11185559/

Récuperation des données et divers script sur la page :
    https://osf.io/kjfbz/
    
    - donnée Raw Data : obtenue avec STARsolo
    GEO060 => M14
    GEO061 => M15
    

# Chargement des librairies

library(reticulate)
library(dplyr)
library(cowplot)
library(Seurat) ##Note this code was written for a previous version of Seurat (4.1.1)
library(ggplot2)
library(Matrix)
library(forcats)
library(RColorBrewer)

# library(akmedoids) # on ne peut pas charger le pac-kages ici car il n'est pas installé sur le serveur et aussi n'est plus compatible avec la version de R


#--Figure 1-----------------------------------------------------------------------------


#fonction pour calculer le taux de croissance
doublingrate <- function(fit, x, t){
  drs <- c()
  for (i in 2:length(fit)){
    dr <- log2(fit[i]/fit[i-1])/(x[i]-x[i-1])
    drs <- c(drs,dr)
  }
  
  grs <- c()
  for (j in 2:length(t)){
    k <- which.min(abs(x - t[j]))
    gr <- drs[k]
    grs <- c(grs, gr)
  }
  return(grs)
}



#bacteria
hour  <- c(0.00,1.10,2.13,2.45,2.75,2.90,3.18,3.28,3.60,3.93,4.0,4.42,4.67,4.95,6.12,6.97) #Kuchina and Brettner et al. 2021 (Science)
OD600 <- c(0.02,0.04,0.27,0.49,0.94,0.93,1.16,1.34,1.58,1.76,2.0,2.87,3.20,3.56,5.31,6.34)



fitb <- nls(OD600 ~ L/(1+exp(-k*(hour - x0))), start = list(L = 6, k = 0.5, x0 = 3))
Lb  <- summary(fitb)$parameters[1,1]
kb  <- summary(fitb)$parameters[2,1]
x0b <- summary(fitb)$parameters[3,1]
xb <- seq(from = 0, to = 8, by = 0.1)
fitb <- Lb/(1+exp(-kb*(xb - x0b)))



GpHb <- doublingrate(fitb,xb,xb)
GpHb <- data.frame(t = xb[-1], gph = GpHb)
GpHb_data <- doublingrate(fitb,xb,hour)
GpHb_data <- data.frame(t = hour[c(4,5,8,9,12,13,14,15)], gph = GpHb_data[c(3,4,7,8,11,12,13,14)])

cell_density <- data.frame(t = xb, cpml = fitb)
cell_dens_data <- data.frame(t = hour[c(4,5,8,9,12,13,14,15)], cpml = OD600[c(4,5,8,9,12,13,14,15)] )

b1.1.1 <- ggplot(GpHb, aes(x = t, y = gph)) +
  geom_line() +
  geom_point(data = GpHb_data, aes(x = t, y = gph), size = 4, fill = "#8CA2CB", shape = 21) +
  ylab("growth rate (generations/hour)") +
  xlab("time (hours)") +
  scale_y_continuous(position = "right") +
  theme(
    panel.background = element_rect(fill = 'transparent'),
    plot.background = element_rect(fill = 'transparent', color = NA),
    panel.grid.major = element_blank(), panel.grid.minor = element_blank(), 
    axis.line = element_line(size = 0.5, color = "black"),
    aspect.ratio = 1,
    text = element_text(size=16),
    axis.title = element_blank()
  ) +
  NoLegend()
b1.1.1

b1.1.2 <- ggplot(cell_density, aes(x = t, y = cpml)) +
  geom_line(color = "gray") +
  geom_point(data = cell_dens_data, aes(x = t, y = cpml), size = 4, fill = "#8CA2CB", shape = 21, color = "gray") +
  ylab("growth rate (generations/hour)") +
  xlab("time (hours)") +
  theme(
    panel.background = element_rect(fill = 'transparent'),
    plot.background = element_rect(fill = 'transparent', color = NA),
    panel.grid.major = element_blank(), panel.grid.minor = element_blank(), 
    axis.line = element_line(size = 0.5, color = "black"),
    aspect.ratio = 1,
    text = element_text(size=16),
    axis.title = element_blank()
  ) +
  NoLegend()
b1.1.2





b1.2 <- ggplot(odRNAb.M15, aes(x = GpH, y = rRNA, fill = factor(GpH))) +
  stat_summary(fun = mean, geom = "point", size = 4, fill = "#8CA2CB", shape = 21) +
  geom_smooth(aes(fill = GpH), method = "lm", se = FALSE, color = "black") +
  ylab("mean rRNA counts/cell") +
  xlab("growth rate (generations/hour)") +
  scale_fill_manual(values = c("#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB")) +
  theme(
    panel.background = element_rect(fill = 'transparent'),
    plot.background = element_rect(fill = 'transparent', color = NA),
    axis.line = element_line(size = 0.5, color = "black"),
    aspect.ratio = 1,
    text = element_text(size=16),
    axis.title = element_blank()
  ) +
  NoLegend()
b1.2

x <- c(mean(OD0.5.RNA$GpH), 
       mean(OD1.0.RNA$GpH), 
       mean(OD1.3.RNA$GpH), 
       mean(OD1.6.RNA$GpH),
       mean(OD2.8.RNA$GpH),
       mean(OD3.6.RNA$GpH),
       mean(OD5.3.RNA$GpH),
       mean(OD6.0.RNA$GpH))

y <- c(mean(OD0.5.RNA$rRNA), 
       mean(OD1.0.RNA$rRNA), 
       mean(OD1.3.RNA$rRNA), 
       mean(OD1.6.RNA$rRNA),
       mean(OD2.8.RNA$rRNA),
       mean(OD3.6.RNA$rRNA),
       mean(OD5.3.RNA$rRNA),
       mean(OD6.0.RNA$rRNA))

summary(lm(y~x))




b1.3 <- ggplot(odRNAb, aes(x = GpH, y = log10(rRNA), fill = factor(GpH))) +
  geom_jitter(aes(color = "#2c4470"), alpha = 0.5, size = 0.01) +
  geom_violin(trim = FALSE, alpha = 0.8) +
  stat_summary(fun = mean, geom = "point", size = 2) +
  geom_smooth(aes(fill = GpH), method = "lm", se = FALSE, color = "black") +
  ylab("log10(rRNA counts/cell)") +
  xlab("growth rate (generations/hour)") +
  scale_fill_manual(values = c("#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB","#8CA2CB")) +
  scale_color_manual(values = c("#2c4470")) +
  theme(
    panel.background = element_rect(fill = 'transparent'),
    plot.background = element_rect(fill = 'transparent', color = NA),
    axis.line = element_line(size = 0.5, color = "black"),
    aspect.ratio = 1,
    text = element_text(size=16),
    axis.title = element_blank()
  ) +
  scale_y_continuous(breaks = c(2,3,4,5),labels = c("100","1,000", "10,000", "100,000")) +
  NoLegend()
b1.3

summary(lm(log10(odRNAb$rRNA)~odRNAb$GpH))



# ce code génère plusieurs graphiques à l’aide de ggplot2. Voici un résumé des graphiques créés dans le code :

# 	1.	Graphique b1.1.1 :
# 	•	Titre : Taux de croissance en fonction du temps.
# 	•	Ce graphique montre le taux de croissance (générations par heure, gph) calculé par la fonction doublingrate en fonction du temps (t), avec une courbe lissée de ce taux de croissance. Des points sont ajoutés pour les valeurs observées à certains moments spécifiques (heure hour).

# 	2.	Graphique b1.1.2 :
# 	•	Titre : Densité cellulaire en fonction du temps.
# 	•	Ce graphique montre la densité cellulaire (mesurée par OD600), qui représente l’absorbance à 600 nm, en fonction du temps (t). Les valeurs ajustées à partir du modèle sigmoïde sont tracées, avec des points indiquant les valeurs réelles mesurées à certains moments du temps.

# 	3.	Graphique b1.2 :
# 	•	Titre : Relation entre le taux de croissance et la quantité d’ARN ribosomique par cellule.
# 	•	Ce graphique examine la relation entre le taux de croissance (en générant GpH) et la quantité d’ARN ribosomique (rRNA) par cellule. Les points moyens (en fonction de GpH) sont visualisés et un modèle linéaire est ajusté pour montrer cette relation.

# 	4.	Graphique b1.3 :
# 	•	Titre : Distribution de log10(rRNA) en fonction du taux de croissance.
# 	•	Ce graphique montre la distribution de la quantité d’ARN ribosomique en utilisant log10(rRNA) en fonction du taux de croissance (GpH). Il combine un graphique de type violon (distribution des données) avec des points de résumé pour la moyenne et une ligne de régression linéaire ajustée.

# Ces graphiques sont générés de manière transparente avec un fond et des axes stylisés, et certains ont des ajustements spécifiques comme l’absence de légende ou une échelle log pour l’axe des ordonnées.




